In [175]:
import pandas as pd


def top_three_salaries(
    employee: pd.DataFrame, department: pd.DataFrame
) -> pd.DataFrame:
    df = (
        employee.merge(
            department.rename(columns={"id": "departmentId", "name": "Department"}),
            on="departmentId",
            how="inner",
        )
        .drop(columns=["departmentId", "id"])
        .rename(columns={"name": "Employee", "salary": "Salary"})
        .filter(items=["Department", "Employee", "Salary"])
    )

    ret = df.head(0)

    for _, gdf in df.groupby("Department"):
        temp = gdf.sort_values(by="Salary", ascending=False)
        unique_salaries = temp["Salary"].drop_duplicates().nlargest(3)
        top_salaries = temp[temp["Salary"].isin(unique_salaries)]
        ret = pd.concat([ret, top_salaries])

    return ret

In [178]:
# Test Code
employee_data = {
    "id": [1, 2, 3, 4, 5, 6, 7],
    "name": ["Joe", "Henry", "Sam", "Max", "Janet", "Randy", "Will"],
    "Salary": [85000, 80000, 60000, 90000, 69000, 85000, 70000],
    "departmentId": [1, 2, 2, 1, 1, 1, 1],
}
employee = pd.DataFrame(employee_data)

# Department DataFrame
department_data = {"id": [1, 2], "name": ["IT", "Sales"]}
department = pd.DataFrame(department_data)

# Running the function
top_salaries = top_three_salaries(employee, department)
top_salaries

,Department,Employee,Salary
0,IT,Joe,85000
1,Sales,Henry,80000
2,Sales,Sam,60000
3,IT,Max,90000
5,IT,Randy,85000
6,IT,Will,70000


In [181]:
mysql = """
WITH ranked AS (
    SELECT 
        d.name AS Department, 
        e.name AS Employee, 
        e.salary AS Salary, 
        dense_rank() OVER (PARTITION BY d.name ORDER BY e.salary DESC) AS salary_rank
    FROM 
        Employee e
    JOIN 
        Department d ON e.departmentId = d.id

) 
SELECT Department, Employee, Salary 
FROM ranked 
WHERE salary_rank <= 3
"""
